In [1]:
from scipy.stats import skew
#from IPython.display import display
#import matplotlib.pyplot as plt
#import seaborn as sns

import glob, re
import numpy as np
import pandas as pd
from sklearn import *
#from sklearn.model_selection import cross_val_score, train_test_split
#from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
#from sklearn.metrics import mean_squared_error, make_scorer
#from sklearn.model_selection import StratifiedKFold, ParameterGrid
import datetime as dt
from datetime import datetime
from xgboost import XGBRegressor
import xgboost as xgb

from keras.layers import Embedding, Input, Dense
import keras
import keras.backend as K

import matplotlib.pyplot as plt
from keras.models import Sequential, Model

/Users/suzukishinji/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/suzukishinji/anaconda/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will 

In [208]:
data = {
'as' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/air_store_info.csv'),
'tra' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/air_visit_data.csv'),
'tes' : pd.read_csv('/Users/suzukishinji/kaggle/recluit/sample_submission-3.csv')}

In [209]:
data['tra'].shape

(252108, 3)

In [210]:
data['as'].shape

(829, 5)

In [211]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

In [212]:
data['tra'].shape

(252108, 6)

In [213]:
data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [214]:
unique_stores = data['tes']['air_store_id'].unique()

In [215]:
data['tra'].shape

(252108, 6)

In [216]:
train = pd.merge(data['tra'], data['as'], how ='left', on = ('air_store_id'))
test = pd.merge(data['tes'], data['as'], how ='left', on = ('air_store_id'))

In [217]:
test.shape

(32019, 11)

In [218]:
test_temp = test['air_area_name'].str.split(' ', expand=True)
test_temp.columns = ['area-1', 'area-2', 'area-3','area-4','area-5']
test = pd.concat([test,test_temp ], axis = 1)
test = test.drop("air_area_name", axis = 1)

In [219]:
train_temp = train['air_area_name'].str.split(' ', expand=True)
train_temp.columns = ['area-1', 'area-2', 'area-3','area-4','area-5']
train = pd.concat([train,train_temp ], axis = 1)
train = train.drop("air_area_name", axis = 1)

In [220]:
train['area-1'].value_counts()

Tōkyō-to         133063
Fukuoka-ken       39645
Ōsaka-fu          22821
Hyōgo-ken         17846
Hokkaidō          13055
Hiroshima-ken      9858
Miyagi-ken         5959
Shizuoka-ken       5798
Niigata-ken        4063
Name: area-1, dtype: int64

In [221]:
tokyo = train[train['area-1'] =='Tōkyō-to' ]
fukuoka = train[train['area-1'] =='Fukuoka-ken' ]
osaka = train[train['area-1'] =='Ōsaka-fu' ]
hyougo = train[train['area-1'] =='Hyōgo-ken' ]
hokkaido = train[train['area-1'] =='Hokkaidō' ]
hiroshima = train[train['area-1'] =='Hiroshima-ken' ]
miyagi = train[train['area-1'] =='Miyagi-ken' ]
shizuoka = train[train['area-1'] =='Shizuoka-ken' ]
niigata = train[train['area-1'] =='Niigata-ken' ]

tokyo_t = test[test['area-1'] =='Tōkyō-to' ]
fukuoka_t = test[test['area-1'] =='Fukuoka-ken' ]
osaka_t = test[test['area-1'] =='Ōsaka-fu' ]
hyougo_t = test[test['area-1'] =='Hyōgo-ken' ]
hokkaido_t = test[test['area-1'] =='Hokkaidō' ]
hiroshima_t = test[test['area-1'] =='Hiroshima-ken' ]
miyagi_t = test[test['area-1'] =='Miyagi-ken' ]
shizuoka_t = test[test['area-1'] =='Shizuoka-ken' ]
niigata_t = test[test['area-1'] =='Niigata-ken' ]

In [222]:
niigata_t.head()

,id,visitors,visit_date,air_store_id,dow,year,month,air_genre_name,latitude,longitude,area-1,area-2,area-3,area-4,area-5
1443,air_0b1e72d2d4422b20_2017-04-23,0,2017-04-23,air_0b1e72d2d4422b20,6,2017,4,Izakaya,37.916367,139.036327,Niigata-ken,Niigata-shi,Gakkōchōdōri,1,Banchō
1444,air_0b1e72d2d4422b20_2017-04-24,0,2017-04-24,air_0b1e72d2d4422b20,0,2017,4,Izakaya,37.916367,139.036327,Niigata-ken,Niigata-shi,Gakkōchōdōri,1,Banchō
1445,air_0b1e72d2d4422b20_2017-04-25,0,2017-04-25,air_0b1e72d2d4422b20,1,2017,4,Izakaya,37.916367,139.036327,Niigata-ken,Niigata-shi,Gakkōchōdōri,1,Banchō
1446,air_0b1e72d2d4422b20_2017-04-26,0,2017-04-26,air_0b1e72d2d4422b20,2,2017,4,Izakaya,37.916367,139.036327,Niigata-ken,Niigata-shi,Gakkōchōdōri,1,Banchō
1447,air_0b1e72d2d4422b20_2017-04-27,0,2017-04-27,air_0b1e72d2d4422b20,3,2017,4,Izakaya,37.916367,139.036327,Niigata-ken,Niigata-shi,Gakkōchōdōri,1,Banchō


In [223]:
tra_tokyo = tokyo[['air_store_id','visit_date','visitors']]
tra_fukuoka = fukuoka[['air_store_id','visit_date','visitors']]
tra_osaka = osaka[['air_store_id','visit_date','visitors']]
tra_hyougo = hyougo[['air_store_id','visit_date','visitors']]
tra_hokkaido = hokkaido[['air_store_id','visit_date','visitors']]
tra_hiroshima = hiroshima[['air_store_id','visit_date','visitors']]
tra_miyagi = miyagi[['air_store_id','visit_date','visitors']]
tra_shizuoka = shizuoka[['air_store_id','visit_date','visitors']]
tra_niigata = niigata[['air_store_id','visit_date','visitors']]

tes_tokyo = tokyo_t[['id','visitors']]
tes_fukuoka = fukuoka_t[['id','visitors']]
tes_osaka = osaka_t[['id','visitors']]
tes_hyougo = hyougo_t[['id','visitors']]
tes_hokkaido = hokkaido_t[['id','visitors']]
tes_hiroshima = hiroshima_t[['id','visitors']]
tes_miyagi = miyagi_t[['id','visitors']]
tes_shizuoka = shizuoka_t[['id','visitors']]
tes_niigata = niigata_t[['id','visitors']]
print(tra_tokyo.shape)
print(tra_fukuoka.shape)
print(tra_osaka.shape)
print(tra_hyougo.shape)
print(tra_hokkaido.shape)
print(tra_hiroshima.shape)
print(tra_miyagi.shape)
print(tra_shizuoka.shape)
print(tra_niigata.shape)

(133063, 3)
(39645, 3)
(22821, 3)
(17846, 3)
(13055, 3)
(9858, 3)
(5959, 3)
(5798, 3)
(4063, 3)


In [224]:
tes_shizuoka.head()

,id,visitors
3315,air_20add8092c9bb51d_2017-04-23,0
3316,air_20add8092c9bb51d_2017-04-24,0
3317,air_20add8092c9bb51d_2017-04-25,0
3318,air_20add8092c9bb51d_2017-04-26,0
3319,air_20add8092c9bb51d_2017-04-27,0


In [225]:
tra_tokyo[['air_store_id', 'visit_date','visitors']].to_csv('tra_tokyo.csv', index = False)
tra_fukuoka[['air_store_id', 'visit_date','visitors']].to_csv('tra_fukuoka.csv', index = False)
tra_osaka[['air_store_id', 'visit_date','visitors']].to_csv('tra_osaka.csv', index = False)
tra_hyougo[['air_store_id', 'visit_date','visitors']].to_csv('tra_hyougo.csv', index = False)
tra_hokkaido[['air_store_id', 'visit_date','visitors']].to_csv('tra_hokkaido.csv', index = False)
tra_hiroshima[['air_store_id', 'visit_date','visitors']].to_csv('tra_hiroshima.csv', index = False)
tra_miyagi[['air_store_id', 'visit_date','visitors']].to_csv('tra_miyagi.csv', index = False)
tra_shizuoka[['air_store_id', 'visit_date','visitors']].to_csv('tra_shizuoka.csv', index = False)

tes_tokyo[['id','visitors']].to_csv('tes_tokyo.csv', index = False)
tes_fukuoka[['id','visitors']].to_csv('tes_fukuoka.csv', index = False)
tes_osaka[['id','visitors']].to_csv('tes_osaka.csv', index = False)
tes_hyougo[['id','visitors']].to_csv('tes_hyougo.csv', index = False)
tes_hokkaido[['id','visitors']].to_csv('tes_hokkaido.csv', index = False)
tes_hiroshima[['id','visitors']].to_csv('tes_hiroshima.csv', index = False)
tes_miyagi[['id','visitors']].to_csv('tes_miyagi.csv', index = False)
tes_shizuoka[['id','visitors']].to_csv('tes_shizuoka.csv', index = False)

In [226]:
area = pd.read_csv('/Users/suzukishinji/kaggle/recluit/tes_tokyo.csv')
area.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


In [141]:
area.shape

(39645, 3)